In [7]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import glob, os, sys, re, math, random
import xml.etree.ElementTree as ET
from rules import *
from predict import *
from raven_data import load_question, display_problem
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import classification_report

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = '../RAVEN/new_data_0'
subtype = '*'
items = sorted([x.split('.xml')[0] for x in glob.glob(os.path.join(path, subtype, '*.xml'))])
debug=False
raw_X = []
raw_y = []
raw_partition = []
if debug:
    items=['../RAVEN/new_data_0/distribute_four/RAVEN_62_train']
for index, item in tqdm(enumerate(items), total=len(items)):
    embeddings, embedding_names, answer = load_question(item, display=debug, debug=False)
    if any(len(embedding) != len(embeddings[0]) for embedding in embeddings):
        print("Error!", item, " has embeddings of different lengths")
        break
    guess, _, y_entry = predict(embeddings,embedding_names,debug=debug)
    x = np.array(embeddings[:8])
    if guess == answer:
        raw_X.append(x)
        raw_y.append(y_entry)
        raw_partition.append([partition_type in item for partition_type in ['train', 'val', 'test']])
    else:
        if debug:
            grid = [[0, 1, 2], [3, 4, 5], [6, 7, None]]
            print("\n".join([str([embeddings[col] if col is not None else None for col in row ]) for row in grid]))
            print("Answer", answer, embeddings[8+answer], "Guess", guess, embeddings[8+guess] if guess is not None else None)
        print("WRONG",item, "Wrong guess", guess, "Correct answer", answer)
print("Using", len(raw_X), "datapoints")

  0%|          | 0/700 [00:00<?, ?it/s]

Using 700 datapoints


In [3]:
raw_X[0], raw_y[0], raw_partition[0]

(array([[0, 0, 3, 1, 9, 3, 1, 1],
        [0, 1, 3, 3, 8, 3, 1, 1],
        [0, 2, 3, 5, 7, 3, 1, 1],
        [1, 0, 1, 1, 5, 0, 1, 1],
        [1, 1, 1, 3, 4, 0, 1, 1],
        [1, 2, 1, 5, 3, 0, 1, 1],
        [2, 0, 4, 0, 5, 4, 1, 1],
        [2, 1, 4, 2, 4, 4, 1, 1]]),
 [4, 1, 0, 0, 0],
 [True, False, False])

In [4]:
max_length = max([x.shape[1] for x in raw_X])
def pad(x):
    return np.pad(x, ((0,0), (0, max_length - x.shape[1])), mode='constant').flatten()
X = np.array([pad(x) for x in raw_X])
y = np.array(raw_y)
partition = np.array(raw_partition)
X.shape,y.shape

((700, 112), (700, 5))

In [5]:
# Split data across train/val/test partitions
X_train, X_val, X_test = X[partition[:, 0]], X[partition[:, 1]], X[partition[:, 2]]
y_train, y_val, y_test = y[partition[:, 0]], y[partition[:, 1]], y[partition[:, 2]]
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((420, 112), (140, 112), (140, 112), (420, 5), (140, 5), (140, 5))

In [8]:
base_model = RandomForestRegressor()
model = MultiOutputRegressor(base_model)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and continuous-multioutput targets

In [12]:
# Use model predictions

path = '../RAVEN/new_data_0'
subtype = 'distribute_four'
items = sorted([x.split('.xml')[0] for x in glob.glob(os.path.join(path, subtype, '*.xml'))])
correct = 0
debug=False
if debug:
    items=['../RAVEN/new_data_0/distribute_four/RAVEN_89_test']
for item in items:
    if 'test' not in item:
        continue
    embeddings, embedding_names, answer = load_question(item, display=False, debug=False)
    if any(len(embedding) != len(embeddings[0]) for embedding in embeddings):
        print("Error!", item, " has embeddings of different lengths")
        break
    x = pad(np.array(embeddings[:8])).reshape(1, -1)
    y_pred = model.predict(x)
    # y_prob = model.predict_proba(x)
    # y_prob_formatted = [y_prob_i[0][1] for y_prob_i in y_prob]
    # print({rule.__name__: prob for rule, prob in zip(rule_order, y_prob_formatted)})
    guess, _, y_entry = predict(embeddings,embedding_names,debug=False)
    print(y_pred)
    print(y_entry)
    if guess == answer:
        correct += 1
        if correct == 10:
            break
    else:
        if debug:
            grid = [[0, 1, 2], [3, 4, 5], [6, 7, None]]
            print("\n".join([str([embeddings[col] if col is not None else None for col in row ]) for row in grid]))
            print("Answer", answer, embeddings[8+answer], "Guess", guess, embeddings[8+guess] if guess is not None else None)
        print("WRONG",item, "Wrong guess", guess, "Correct answer", answer)
print("Final score:", correct, "out of", len(items))

[[1.27 0.99 0.2  0.56 1.52]]
[1, 1, 1, 1, 1]
[[2.85 0.67 0.39 0.7  0.59]]
[2, 1, 0, 2, 0]
[[0.94 0.83 0.28 0.54 1.89]]
[0, 1, 1, 1, 3]
[[1.77 0.59 0.29 0.77 0.67]]
[3, 0, 0, 2, 0]
[[1.12 0.95 0.51 0.52 1.02]]
[0, 0, 0, 0, 0]
[[1.44 0.77 0.29 1.05 1.38]]
[0, 0, 0, 1, 3]
[[1.03 0.8  0.46 1.03 1.3 ]]
[0, 2, 0, 0, 3]
[[2.36 0.69 0.31 0.89 1.52]]
[1, 2, 1, 0, 1]
[[2.34 0.76 0.47 0.8  1.56]]
[0, 0, 3, 0, 3]
[[1.62 0.71 0.58 0.88 1.69]]
[3, 0, 1, 2, 0]
Final score: 10 out of 100
